## Import All Required Libraries

In [1]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [2]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Load Initial Training Data

In [3]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

# load training data
new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[0:25]
new_batch.head()

,playlist_uri,playlist_name
21,spotify:playlist:01clVI1RPrF8zHMxrqVIs0,2020 top rap and hiphop hits 2022
34,spotify:playlist:02eILNCPJV0hoYxMUreT0L,He is back
40,spotify:playlist:02ReQp580O4ormFpFP6CV1,COUNTRY HITS 2021??
41,spotify:playlist:02s8f8AJSucHqFAoQEiIYQ,VOLLEYBALL HYPE (100% clean)
42,spotify:playlist:02t75h5hsNOw4VlC1Qad9Z,Country Party Songs ?? ??


## Iterate Through Training Data to Change Grain of Data

In [4]:
load_batch = []
series = new_batch['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                load_batch.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                load_batch.append(tracks)
    except:
            pass
load_batch = pd.concat(load_batch)
load_batch.head()

100%|██████████| 25/25 [00:24<00:00,  1.03it/s]


,added_at,is_local,primary_color_x,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_external_urls_spotify,...,owner_href,owner_id,owner_type,owner_uri,tracks_href,tracks_limit,tracks_next,tracks_offset,tracks_previous,tracks_total
0,2020-04-25T01:13:53Z,False,None,https://open.spotify.com/user/gracievs,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,album,https://open.spotify.com/album/7DYy5TiODoqnZbZ...,...,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,https://api.spotify.com/v1/playlists/02ReQp580...,100,https://api.spotify.com/v1/playlists/02ReQp580...,0,None,196
1,2020-04-25T01:16:07Z,False,None,https://open.spotify.com/user/gracievs,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,compilation,https://open.spotify.com/album/7pul7RvuYLqryKe...,...,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,https://api.spotify.com/v1/playlists/02ReQp580...,100,https://api.spotify.com/v1/playlists/02ReQp580...,0,None,196
2,2020-04-25T01:17:31Z,False,None,https://open.spotify.com/user/gracievs,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,album,https://open.spotify.com/album/0AXoQGOZDaYSaOo...,...,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,https://api.spotify.com/v1/playlists/02ReQp580...,100,https://api.spotify.com/v1/playlists/02ReQp580...,0,None,196
3,2020-04-25T01:17:48Z,False,None,https://open.spotify.com/user/gracievs,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,album,https://open.spotify.com/album/1UrEMAvTDJdsJsZ...,...,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,https://api.spotify.com/v1/playlists/02ReQp580...,100,https://api.spotify.com/v1/playlists/02ReQp580...,0,None,196
4,2020-04-25T01:18:11Z,False,None,https://open.spotify.com/user/gracievs,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,album,https://open.spotify.com/album/4ybWq0AegFFAvHU...,...,https://api.spotify.com/v1/users/gracievs,gracievs,user,spotify:user:gracievs,https://api.spotify.com/v1/playlists/02ReQp580...,100,https://api.spotify.com/v1/playlists/02ReQp580...,0,None,196


## Upload Training Data to playlist_tracks Table

In [5]:
load_batch.to_sql('playlist_tracks', engine, if_exists='append')

357

## Identify All Artists in Every Song

In [6]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)

outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

  0%|          | 0/3660 [00:00<?, ?it/s]

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,蒋雪儿,spotify:track:007ibBncyX6qXsy3sSCpgV,1.0
0,Avril Lavigne,spotify:track:00Mb3DuaIH1kjrwOku9CGU,1.0
0,2 Chainz,spotify:track:00QyLmjxaSEE8qIZQjBXBj,1.0
1,Wiz Khalifa,spotify:track:00QyLmjxaSEE8qIZQjBXBj,2.0
0,DJ TikTok,spotify:track:01c4DytqvZfRYllvN4BiTB,1.0
...,...,...,...
0,Lil Mosey,spotify:track:7zLYKWcXnYeHHWidalz7rj,1.0
0,Grant,spotify:track:7zPAsqqz9M5qcpG42YUiug,1.0
1,Juneau,spotify:track:7zPAsqqz9M5qcpG42YUiug,2.0
0,Tiësto,spotify:track:7zWJ6Yu8DlIFxEANCkmsBb,2.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
df2

RN,track_uri,artist1,artist2,artist3,artist4,artist5
0,spotify:track:007ibBncyX6qXsy3sSCpgV,蒋雪儿,NaN,NaN,NaN,NaN
1,spotify:track:00Mb3DuaIH1kjrwOku9CGU,Avril Lavigne,NaN,NaN,NaN,NaN
2,spotify:track:00QyLmjxaSEE8qIZQjBXBj,2 Chainz,Wiz Khalifa,NaN,NaN,NaN
3,spotify:track:01c4DytqvZfRYllvN4BiTB,DJ TikTok,NaN,NaN,NaN,NaN
4,spotify:track:01stPT7J3W6Zx45jj1f4nk,Ayo & Teo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1398,spotify:track:7zKncUwLtKCMzwhEb3Jiqd,RIOT,NaN,NaN,NaN,NaN
1399,spotify:track:7zLYKWcXnYeHHWidalz7rj,Lil Mosey,NaN,NaN,NaN,NaN
1400,spotify:track:7zPAsqqz9M5qcpG42YUiug,Grant,Juneau,NaN,NaN,NaN
1401,spotify:track:7zWJ6Yu8DlIFxEANCkmsBb,Hardwell,Tiësto,NaN,NaN,NaN


## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

403

## Get All Audio Features

In [ ]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks''', engine)

outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
df = pd.Series(anti_join['track_uri'])

base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)
df3

100%|██████████| 1404/1404 [02:07<00:00, 10.99it/s]


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:55n9yjI6qqXh5F2mYvUc2y,0.731,0.445,0,-8.418,1,0.0533,0.05860,0.000020,0.1400,0.1060,117.970,247160,4
0,spotify:track:3FrX3mx8qq7SZX2NYbzDoj,0.675,0.601,11,-4.733,0,0.1160,0.00372,0.000000,0.0458,0.9310,127.938,201040,4
0,spotify:track:4TIqzdAssasqx3DAe6cG9J,0.597,0.571,1,-6.205,0,0.2930,0.49200,0.000000,0.0930,0.1650,106.196,218448,3
0,spotify:track:3E7dfMvvCLUddWissuqMwr,0.652,0.698,10,-4.667,0,0.0420,0.00112,0.000115,0.0886,0.4700,96.021,202067,4
0,spotify:track:5Nz2BnJroLw1HPDeynxWGx,0.540,0.929,1,-3.830,1,0.4470,0.00183,0.000193,0.2340,0.4490,75.029,156800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,spotify:track:40uMIn2zJLAQhNXghRjBed,0.797,0.631,3,-3.818,0,0.0786,0.09040,0.000004,0.0998,0.2880,129.915,184213,4
0,spotify:track:7k6IzwMGpxnRghE7YosnXT,0.803,0.454,8,-4.802,0,0.0294,0.35200,0.000000,0.0655,0.7390,99.990,192213,4
0,spotify:track:6I7FFkcXS58luQoYLzPHi3,0.683,0.779,9,-6.324,1,0.0546,0.02430,0.839000,0.1190,0.1360,103.004,219818,4
0,spotify:track:1PCnL3pZc3czpIngzKbSBE,0.487,0.661,1,-6.469,0,0.0360,0.00196,0.717000,0.1110,0.0463,97.042,259794,4


## Upload Audio Features to Database

In [ ]:
df3.to_sql('audio_features', con=engine, if_exists='append')

403

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT pt.*, artist1, artist2, artist3, artist4, artist5, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN audio_features af ON pt.track_uri = af.track_uri
JOIN artists a ON pt.track_uri = a.track_uri''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+")